This was a code to locally train the deep learning model but it takes too long.

In [3]:
import numpy as np
import tifffile
from supercloudUtils import train_model, segment_image
import matplotlib.pyplot as plt
import torch
from time import perf_counter
import warnings 
warnings.filterwarnings('ignore')

In [4]:
data_path = "/Users/tthomas/Desktop/ESS569/MLGEO2024_MarsFans/data/ai_ready/" # This is an example, change it to your own path

i = 0
tic = perf_counter()
for ctxIMG in range(2):
    for ctxDEM in range(2):
        for ctxSLOPE in range(2):
                tic_inner = perf_counter()
                name_list = []
                training_img = []
                if ctxIMG:
                    img = tifffile.imread(data_path + "ctxIMG_aiready.tif")
                    training_img.append(img)
                    name_list.append('ctxIMG')
                if ctxDEM:
                    img = tifffile.imread(data_path + "ctxDEM_aiready.tif")
                    training_img.append(img)
                    name_list.append('ctxDEM')
                if ctxSLOPE:
                    img = tifffile.imread(data_path + "ctxSLOPE_aiready.tif")
                    training_img.append(img)
                    name_list.append('ctxSLOPE')

                if ~np.any(training_img): # handle the case where no data is selected
                                        continue

                s = np.shape(training_img) # need at least 2 channels
                if s[0] <= 1:
                    continue

                training_img = np.transpose(training_img,(1,2,0))
                saveModelPath = '/Users/tthomas/Desktop/ESS569/MLGEO2024_MarsFans/results/CTX_only/models/' + '_'.join(name_list) + '.pth'
                saveResultPath = '/Users/tthomas/Desktop/ESS569/MLGEO2024_MarsFans/results/CTX_only/images/' + '_'.join(name_list) + '.png'
                model = train_model(training_img) # Train model
                torch.save(model,saveModelPath) # Save model
                #model = torch.load(saveModelPath)
                result = segment_image(training_img, model) # Segment image
                plt.imsave(saveResultPath,result) # Save segmented image

                toc_inner = perf_counter()
                i = i+1
                print("Iteration ",i," complete in %.2f minutes" % round((toc_inner-tic_inner)/(60),2))

                trainingEpochs = 4
                batchSize = 8
                # finalFeatures_grid = [4,6,8,10]
                # patchSize_grid = [448,336,224]
                finalFeatures_grid = [8]
                patchSize_grid = [336]
                
                for finalFeatures in finalFeatures_grid:
                    for patchSize in patchSize_grid:
                        tic_inner = perf_counter()
                        
                        hyper_str = "_ff" + str(finalFeatures) + "_ps" + str(patchSize)
                        
                        saveModelPath = '/Users/tthomas/Desktop/ESS569/MLGEO2024_MarsFans/results/CTX_only/models/' + '_'.join(name_list) + hyper_str + '.pth'
                        saveResultPath = '/Users/tthomas/Desktop/ESS569/MLGEO2024_MarsFans/results/CTX_only/images/' + '_'.join(name_list) + hyper_str + '.png'
                        
                        model = train_model(training_img,trainingEpochs=trainingEpochs,batchSize=batchSize,
                                            finalFeatures=finalFeatures,patchSize=patchSize) # Train model
                        torch.save(model,saveModelPath) # Save model
                        #model = torch.load(saveModelPath)
                        result = segment_image(training_img, model) # Segment image
                        plt.imsave(saveResultPath,result) # Save segmented image

                        toc_inner = perf_counter()
                        i = i+1
                        print("Iteration ",i," complete in %.2f minutes" % round((toc_inner-tic_inner)/(60),2))

toc = perf_counter()
print("Iterations:",i)
print("Elapsed: %.2f s" % round((toc-tic),2))
print("Elapsed: %.2f min" % round((toc-tic)/60,2))
print("Elapsed: %.2f hr" % round((toc-tic)/(60*60),2))
print("Elapsed: %.2f day" % round((toc-tic)/(60*60*24),2))
print("Minutes per iteration: %.2f" % round(((toc-tic)/(60))/i,2))

KeyboardInterrupt: 